In [3]:
import spacy
from spacy.tokens import Doc, Span
nlp = spacy.load('en_core_web_trf')
nlp.add_pipe('dbpedia_spotlight', config={'confidence': 0.5})
preprocessing = spacy.load('en_core_web_trf')
from fastcoref import LingMessCoref
coref_model = LingMessCoref()
from SPARQLWrapper import SPARQLWrapper, JSON
from datasets import load_dataset
dataset = load_dataset("potsawee/wiki_bio_gpt3_hallucination")
import os
from tqdm.notebook import tqdm
import concurrent.futures
import re
import requests
import pandas as pd

06/25/2024 16:38:31 - INFO - 	 missing_keys: []
06/25/2024 16:38:31 - INFO - 	 unexpected_keys: []
06/25/2024 16:38:31 - INFO - 	 mismatched_keys: []
06/25/2024 16:38:31 - INFO - 	 error_msgs: []
06/25/2024 16:38:31 - INFO - 	 Model Parameters: 590.0M, Transformer: 434.6M, Coref head: 155.4M


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

06/25/2024 12:09:35 - INFO - 	 Use pytorch device_name: mps
06/25/2024 12:09:35 - INFO - 	 Load pretrained SentenceTransformer: all-mpnet-base-v2
/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import numpy as np
from urllib.parse import urlparse

In [5]:
def subjects_direct_links(subjects_set):
    subject_direct_dict = {}
    for subj in subjects_set:
        sparql = SPARQLWrapper("https://dbpedia.org/sparql")
        sparql.setMethod('POST')  

        # Dynamically insert the subject into the query
        query_source_to_target = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbr: <http://dbpedia.org/resource/>

        SELECT DISTINCT ?label
        WHERE {{
        {{
            <{subj}> ?property ?entity .
            ?entity rdfs:label ?label .
        }} UNION {{
            ?entity ?property <{subj}> .
            ?entity rdfs:label ?label .
        }}
        FILTER (lang(?label) = "en")
        }}
        """
        sparql.setQuery(query_source_to_target)
        sparql.setReturnFormat(JSON)

        try:
            result = sparql.query().convert()
            tmp = [item['label']['value'] for item in result["results"]["bindings"]]
            subject_direct_dict[subj] = model.encode(tmp,show_progress_bar=False)
        except Exception as e:
            print(f"Query for {subj} didn't work: {e}")

    return subject_direct_dict


In [6]:
set = {'http://dbpedia.org/resource/Nigel_Milsom'}
subject_dict = subjects_direct_links(set)
embeddings = subject_dict["http://dbpedia.org/resource/Nigel_Milsom"]

In [ ]:
word = "Psychologist"
entity_embedding = model.encode(word)
cosine_similarities = np.dot(embeddings, entity_embedding)
print(np.argmax(cosine_similarities))
cosine_similarities[np.argmax(cosine_similarities)]

In [5]:
def get_subject_spans(sentence):
    # Parse the sentence using spaCy
    doc = preprocessing(sentence)
    
    # Extract subject spans based on dependency labels
    subject_spans = []
    for token in doc:
        if token.dep_ in ("nsubj", "nsubjpass"):
            # Get the span for the subject token
            span = doc[token.left_edge.i : token.right_edge.i + 1]
            subject_spans.append(span.text)
    
    return subject_spans

# Example sentences
sentences = [
    "Albert Einstein was a theoretical physicist who developed the theory of relativity.",
    "The theory of relativity was developed by Albert Einstein.",
    "She enjoys reading books.",
    "The quick brown fox jumps over the lazy dog."
]

# Extract and print subject spans for each sentence
for sentence in sentences:
    subject_spans = get_subject_spans(sentence)
    print(f"Sentence: {sentence}")
    print(f"Subject Spans: {subject_spans}\n")

ValueError: Only numpy and cupy arrays are supported, but found <class 'numpy.ndarray'> instead. If get_array_module module wasn't called directly, this might indicate a bug in Thinc.

In [1]:
nlp_trf = spacy.load("en_core_web_trf")

NameError: name 'spacy' is not defined